In [1]:
import pandas as pd
import numpy as np
import collections

Exercise Break: What is the expected number of occurrences of a 9-mer in 500 random DNA strings, each of length 1000? Assume that the sequences are formed by selecting each nucleotide (A, C, G, T) with the same probability (0.25).

In [2]:
(1000-9+1)*500/(4**9)

1.89208984375

# entropy

In [3]:
# entropy
Pa=[.2,.2,0,0,0,0,.9,.1,.1,.1,.3,0]
Pc=[.1,.6,0,0,0,0,0,.4,.1,.2,.4,.6]
Pg=[0,0,1,1,.9,.9,.1,0,0,0,0,0]
Pt=[.7,.2,0,0,.1,.1,0,.5,.8,.7,.3,.4]
tmp2=0
for i in range(len(Pa)):
    for P in [Pa,Pc,Pg,Pt]:
        if P[i]>0:
            tmp=-1*P[i]*np.log2(P[i])
        else:
            tmp=0
        tmp2+=tmp
tmp2

9.916290005356972

Furthermore, the Frequent Words Problem is inadequate because it does not correctly model the biological problem of motif finding. A DnaA box is a pattern that clumps, or appears frequently, within a relatively short interval of the genome. In contrast, a regulatory motif is a pattern that appears at least once (perhaps with variation) in each of many different regions that are scattered throughout the genome.

# A brute force algorithm for motif finding

Given a collection of strings Dna and an integer d, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most d mismatches. For example, the implanted 15-mer in the strings above represents a (15,4)-motif.


Input: A collection of strings Dna, and integers k and d.
Output: All (k, d)-motifs in Dna.

MotifEnumeration(Dna, k, d)
    Patterns ← an empty set
    for each k-mer Pattern in the first string in Dna
        for each k-mer Pattern’ differing from Pattern by at most d mismatches
            if Pattern' appears in each string from Dna with at most d mismatches
                add Pattern' to Patterns
    remove duplicates from Patterns
    return Patterns

In [4]:
def HammingDistance(texta,textb):
    mis=0
    for i in range(min(len(texta),len(textb))):
        if texta[i]!=textb[i]:
            mis+=1
        else:
            continue
    return mis

def Neighbors(Pattern,d):
    if d == 0:
        return Pattern
    if len(Pattern) == 1: 
        return {'A', 'C', 'G', 'T'}
#     Neighborhood=set()
    Neighborhood=[]
    SuffixNeighbors = Neighbors(Pattern[1:], d)
    for c in SuffixNeighbors:
        if HammingDistance(Pattern[1:], c)<d:
            for base in ['A','T','C','G']:
                Neighborhood.append(base+c)
        else:
            Neighborhood.append(Pattern[0]+c)
    return Neighborhood


def MotifEnumeration(DNA, k, d):
    Patterns =[]
    for dna in DNA:
        dp=[]
        for i in range(len(dna)-k+1):
            pattern=dna[i:i+k]
            neighborhood=Neighbors(pattern,d)
            for j in neighborhood:
                if j not in dp:
                    dp.append(j)
        Patterns.append(dp)
    return Patterns

In [5]:
k=5
d=1
DNA='CAACAGATAAGACATAGTAACCAGG ATGCATAACAAATAATATGGCCGCC TTCCAATTCTAATAAAGACGATGCC CGGTCGATAAGGCATGCCTCCAACA CTTGTCTCATTATAACGGACAAGTT CATAAACTTTCTCCCTTACGACATC'.split(' ')
a=MotifEnumeration(DNA, k, d)
## since output is a 2d list
a=sum(a, []) # flatten into 1D
a=collections.Counter(a) # and count the words that appear in every DNA
keys=[k for k,v in a.items() if v==len(DNA)]
print(*keys)

CAACT AATAA TATAA CATAA GATAA GACAA ATAAA ATAAG ATAAT ATAAC ATGAC CTAAC TAACG


# Median String

Median String Problem: Find a median string.

Input: A collection of strings Dna and an integer k.
Output: A k-mer Pattern that minimizes d(Pattern, Dna) among all possible choices of k-mers.

MedianString(Dna, k)
    distance ← ∞
    for each k-mer Pattern from AA…AA to TT…TT
        if distance > d(Pattern, Dna)
             distance ← d(Pattern, Dna)
             Median ← Pattern
    return Median

In [6]:
from itertools import product
def MedianString(DNA, k):
    distance=10000
    combs = [''.join(comb) for comb in product(['A','T','C','G'], repeat=k)]
    for kmer in combs:
        dp=[]
        for dna in DNA:   
            tmpscore=10000
            for i in range(len(dna)-k+1):
                pattern=dna[i:i+k]
                if HammingDistance(kmer,pattern) <tmpscore:
                    tmpscore=HammingDistance(kmer,pattern)
            dp.append(tmpscore)
        if sum(dp)<distance:
            distance=sum(dp)
            Median=kmer
    return Median

In [7]:
k=7
DNA=['CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC',
'GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC','GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG']
MedianString(DNA, k)

'AATCCTA'

# Profile-most Probable k-mer

Profile-most Probable k-mer Problem: Find a Profile-most probable k-mer in a string.

Input: A string Text, an integer k, and a 4 × k matrix Profile.

Output: A Profile-most probable k-mer in Text.

In [9]:
def ProfileMostProbableKmer(Text,k,prof):
    nuc_dic={'A':0,'C':1,'G':2,'T':3}
    score=0
    bestkmer=[]
    for i in range(len(Text)-k+1):
        tmpscore=1
        pattern=Text[i:i+k]
        for j,p in enumerate(pattern):
            tmpscore=prof[nuc_dic[p]][j]*tmpscore
        if tmpscore>score:
            score=tmpscore
            bestkmer=pattern
        if score==0:
            bestkmer=Text[0:k]
    return bestkmer


def read_file(filename):
    with open(filename, "r") as dataset:
        data = []
        for line in dataset:
            data.append(line.strip())
        Text = data[0]
        k = int(data[1])
        raw_profile = data[2:]
        bases = ['A', 'C', 'G', 'T']
        prof = [list(map(float, raw_profile[i].split())) for i in range(len(raw_profile))]
        prof_array=np.array(prof)
    return Text,k,prof_array
Text,k,prof=read_file('C:/Users/Jofan/Downloads/dataset_159_3.txt')
ProfileMostProbableKmer(Text,k,prof)

'ATTAGGGTGGTTG'

# greedy

GreedyMotifSearch(Dna, k, t)
    BestMotifs ← motif matrix formed by first k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            form Profile from motifs Motif1, …, Motifi - 1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs

In [23]:
def greedy_motif_search(DNAs: list, k_mer_len: int, num_of_DNAs: int) -> list:
    """
    Returns a list of best fitting k_mers for given list of DNAs
    :param DNAs: list of DNA
    :param k_mer_len: Length of k_mer
    :param num_of_DNAs: Number of DNAs
    :return: The list of best fitting k_mers
    """
    best_motifs = [DNA[0:k_mer_len] for DNA in DNAs]
    for i in range(len(DNAs[0]) - k_mer_len + 1):
        motif = DNAs[0][i: i + k_mer_len]
        motifs = [motif]
        profile0=gen_profile_matrix(best_motifs)
        for i in range(1, num_of_DNAs):
            profile = gen_profile_matrix(motifs)
            prob_k_mer = ProfileMostProbableKmer(DNAs[i], k_mer_len, profile)
            motifs.append(prob_k_mer)
        if gen_score(motifs,profile) < gen_score(best_motifs,profile0):
            best_motifs = motifs
    return best_motifs

def gen_profile_matrix(motif_matrix: list, pseudocount=False) -> np.ndarray:
    """
    Returns the count matrix of a motif matrix
    :param motif_matrix: Provided motif matrix
    :return: The count matrix
    """
    seq_len=len(motif_matrix[0])
    num_frag=len(motif_matrix)
    motif_matrix = [element for element in motif_matrix if (element is not None)]
    nucleo_dict = {'A': 0, 'C': 1,
                   'G': 2, 'T': 3}
    count_arr = np.zeros((4, seq_len))

    for col in range(seq_len):
        nucleo_col = list()
        for row in range(num_frag):
            nucleo_col.append(motif_matrix[row][col])
        for nucleo in nucleo_dict.keys():
            occur = nucleo_col.count(nucleo)
            count_arr[nucleo_dict[nucleo], col] = occur
    return count_arr/num_frag
        
#         for nucleo in nucleo_dict.keys():
#             if pseudocount:
#                 occur = nucleo_col.count(nucleo)+1
#             else:
#                 occur = nucleo_col.count(nucleo)
#             count_arr[nucleo_dict[nucleo], col] = occur
#         count_arr_norm=count_arr/sum(count_arr)
#     return count_arr_norm
def gen_consensus_sequence(profile_matrix: np.ndarray) -> str:

    row_nucleo_dict = {0: 'A', 1: 'C',
                       2: 'G', 3: 'T'}

    seq = str()
    nucleotide = str()
    col_length = len(profile_matrix[0, :])
    row_length = len(profile_matrix[:, 0])
    for j in range(col_length):
        max_val = 0
        for i in range(row_length):
            if profile_matrix[i, j] > max_val:
                max_val = profile_matrix[i, j]
                nucleotide = row_nucleo_dict[i]
        seq = seq + nucleotide
    return seq

def gen_score(motifs,profile_matrix):
    consensus=gen_consensus_sequence(profile_matrix)
    score = 0
    for motif in motifs:
        score += HammingDistance(consensus, motif)
    return score

def read_file(filename):
    with open(filename, "r") as dataset:
        data = []
        for line in dataset:
            data.append(line.strip())
        k = data[0].split(' ')[0]
        num_of_DNAs = data[0].split(' ')[1]        
        DNA = data[1].split(' ')
    return DNA,int(k),int(num_of_DNAs)

filename='C:/Users/Jofan/Downloads/dataset_159_5 (1).txt'
DNA,k,num_of_DNAs= read_file(filename)

a=greedy_motif_search(DNA,k,num_of_DNAs)
print(*a)

ACTTCGGAATAC GGGGCTCCCTAG TTTTAAGGGTCC TCGGCTCGATAG GGGGCTGGCTAC TCGGCTCGCTAG GAAGTTACGTCG GGCTGTGTAGGA TAGTGTGGATGG TTGGCTCACTAG ATGGCTCAGTAG ACGGCTCTATAG TGGGCTCTATAG ACTGGTGCCTGG AGGGCTCAATAG TTGGCTCCCTAG GGGGCTCAATAG AGGGCTCACTAG TTAGTTCAATGC ACGGCTCGATAG GTGGCTCTATAG TCAGCACTATCA GGAGCTAACGGG GGGGCTCCGTAG GAGGCTCAATAG


In [20]:
k=3
num_of_DNAs=5
DNA=['GGCGTTCAGGCA','AAGAATCAGTCA','CAAGGAGTTCGC','CACGTCAATCAC','CAATAATATTCG']
tmp=greedy_motif_search(DNA,k,num_of_DNAs)
print(tmp)
gen_profile_matrix(tmp)

['CAG', 'CAG', 'CAA', 'CAA', 'CAA']


<ipython-input-19-c04469168198>:45: RuntimeWarning: invalid value encountered in true_divide
  count_arr_norm=count_arr/sum(count_arr)


array([[0. , 1. , 0.6],
       [1. , 0. , 0. ],
       [0. , 0. , 0.4],
       [0. , 0. , 0. ]])